### [Entendiendo como usar memoria con langgraph](https://langchain-ai.github.io/langgraph/concepts/persistence/#replay)

In [43]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: int
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}

In [44]:
workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)

{'foo': 'b', 'bar': ['a', 'b']}

In [45]:
# get the latest state snapshot
config = {"configurable": {"thread_id": "1"}}
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb98-674e-8002-3e057d4b74f3'}}, metadata={'source': 'loop', 'writes': {'node_b': {'foo': 'b', 'bar': ['b']}}, 'thread_id': '1', 'step': 2, 'parents': {}}, created_at='2025-01-10T17:04:21.684397+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb97-6556-8001-030fd0af2844'}}, tasks=())

In [46]:
# get a state snapshot for a specific checkpoint_id
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1ef663ba-28fe-6528-8002-5a559208592c"}}
graph.get_state(config)

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1ef663ba-28fe-6528-8002-5a559208592c'}}, metadata=None, created_at=None, parent_config=None, tasks=())

In [47]:
config = {"configurable": {"thread_id": "1"}}
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb98-674e-8002-3e057d4b74f3'}}, metadata={'source': 'loop', 'writes': {'node_b': {'foo': 'b', 'bar': ['b']}}, 'thread_id': '1', 'step': 2, 'parents': {}}, created_at='2025-01-10T17:04:21.684397+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb97-6556-8001-030fd0af2844'}}, tasks=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb97-6556-8001-030fd0af2844'}}, metadata={'source': 'loop', 'writes': {'node_a': {'foo': 'a', 'bar': ['a']}}, 'thread_id': '1', 'step': 1, 'parents': {}}, created_at='2025-01-10T17:04:21.683941+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb93-6cda-8000-d3f048d5c0e8'}}, tasks=(Preg

In [48]:
## Se puede actualizar el estado para bifurcar o actualizar el estado

# Obtener el estado más reciente del grafo
config = {"configurable": {"thread_id": "1"}}
state = graph.get_state(config)
print(state)

# Actualizar el estado manualmente
config = {"configurable": {"thread_id": "1"}}
graph.update_state(config, {"foo": 2, "bar": ["b"]})

# Obtener el nuevo estado
updated_state = graph.get_state(config)
print(updated_state)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb98-674e-8002-3e057d4b74f3'}}, metadata={'source': 'loop', 'writes': {'node_b': {'foo': 'b', 'bar': ['b']}}, 'thread_id': '1', 'step': 2, 'parents': {}}, created_at='2025-01-10T17:04:21.684397+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb97-6556-8001-030fd0af2844'}}, tasks=())
StateSnapshot(values={'foo': 2, 'bar': ['a', 'b', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fc07-6c16-8003-a3c231ca542d'}}, metadata={'source': 'update', 'writes': {'node_b': {'foo': 2, 'bar': ['b']}}, 'thread_id': '1', 'step': 3, 'parents': {}}, created_at='2025-01-10T17:04:21.729978+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf74e-fb98-674e-8002-3e057d4b74f3'}}, tasks=())


In [49]:
graph.update_state(config, {"foo": 2, "bar": ["b"]})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efcf74e-fc10-6cb2-8004-64ecb62429f6'}}

### Uso de langgraph

In [56]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore

# Crear el checkpointer para guardar estados
checkpointer = MemorySaver()

# Crear un almacén en memoria
in_memory_store = InMemoryStore()


In [57]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# Estado del grafo
class State(TypedDict):
    messages: list

def node_greet(state: State):
    return {"messages": state["messages"] + ["Hello, how can I assist you?"]}

# Crear y conectar el grafo
graph = StateGraph(State)
graph.add_node(node_greet)
graph.add_edge(START, "node_greet")
graph.add_edge("node_greet", END)


In [58]:
graph = graph.compile(checkpointer=checkpointer, store=in_memory_store)

In [59]:
user_id = "1"
config = {"configurable": {"thread_id": "1", "user_id": user_id}}

# Ejecutar el grafo
for update in graph.stream(
    {"messages": [{"role": "user", "content": "hi"}]}, config, stream_mode="updates"
):
    print(update)


{'node_greet': {'messages': [{'role': 'user', 'content': 'hi'}, 'Hello, how can I assist you?']}}
